In [94]:
import numpy as np
from sklearn import linear_model
import pandas as pd
from sklearn import preprocessing
import datetime

In [33]:
save_dir='../primary_data/'

In [83]:
dems = pd.read_csv('/home/jasonmellone/Documents/nyu/DSQF/stitched_data/county_dems.csv')
dems.primary_date = pd.to_datetime(dems.primary_date)

In [84]:
reps = pd.read_csv('/home/jasonmellone/Documents/nyu/DSQF/stitched_data/county_reps.csv')
reps.primary_date = pd.to_datetime(dems.primary_date)

In [72]:
le = preprocessing.LabelEncoder()

In [73]:
le.fit(list(set(dems['1st'].unique()).union(dems['2nd'].unique()).union(dems['3rd'].unique())))

LabelEncoder()

In [74]:
dems['1st_encoded'] = le.transform(dems['1st'].values)

In [75]:
dems['2nd_encoded'] = le.transform(dems['2nd'].values)

In [76]:
dems['3rd_encoded']= le.transform(dems['3rd'].values)

In [95]:
clf = linear_model.SGDClassifier(loss='hinge')
Xs = ['votes1','votes2','votes3','pct1','pct2','pct3','primary_type','edge1','edge2','edge3',
      'Abbreviation','area_name','state_abbreviation','PST045214','PST040210','PST120214',
      'POP010210','AGE135214','AGE295214','AGE775214','SEX255214','RHI125214','RHI225214',
      'RHI325214','RHI425214','RHI525214','RHI625214','RHI725214','RHI825214','POP715213',
      'POP645213','POP815213','EDU635213','EDU685213','VET605213','LFE305213','HSG010214',
      'HSG445213','HSG096213','HSG495213','HSD410213','HSD310213','INC910213','INC110213',
      'PVY020213','BZA010213','BZA110213','BZA115213','NES010213','SBO001207','SBO315207',
      'SBO115207','SBO215207','SBO515207','SBO415207','SBO015207','MAN450207','WTN220207',
      'RTN130207','RTN131207','AFN120207','BPS030214','LND110210','POP060210']

y_first = '1st_encoded'
y_second = '2nd_encoded'

for dt in sorted(dems.primary_date.unique()):
    if not dt==np.datetime64('NaT'):
        
        dems_past = dems[dems.primary_date<dt].copy()
        dems_current = dems[dems.primary_date==dt].copy()
        
    

2016-02-01T00:00:00.000000000
2016-02-09T00:00:00.000000000
2016-02-20T00:00:00.000000000
2016-02-27T00:00:00.000000000
2016-03-01T00:00:00.000000000
2016-03-05T00:00:00.000000000
2016-03-06T00:00:00.000000000
2016-03-08T00:00:00.000000000
2016-03-15T00:00:00.000000000
2016-03-22T00:00:00.000000000
2016-03-26T00:00:00.000000000
2016-04-05T00:00:00.000000000
2016-04-09T00:00:00.000000000
2016-04-19T00:00:00.000000000
2016-04-26T00:00:00.000000000
2016-05-03T00:00:00.000000000
2016-05-10T00:00:00.000000000
2016-05-17T00:00:00.000000000
2016-05-24T00:00:00.000000000
2016-06-07T00:00:00.000000000


/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: In the future, 'NAT < x' and 'x < NAT' will always be False.
  """Entry point for launching an IPython kernel.
/home/jasonmellone/anaconda2/envs/py3dsqf/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: In the future, 'NAT == x' and 'x == NAT' will always be False.
  


In [85]:
clf.fit(X, Y)

NameError: name 'X' is not defined